In [43]:
from influxdb import InfluxDBClient

import ujson
import copy
import numpy as np
import scipy.stats
import sys

time: 12.3 ms


In [2]:
# Testing auto adding time output to all cells
'''
#To install...
wget https://raw.githubusercontent.com/cpcloud/ipython-autotime/master/autotime.py
#Make available via
jupyter nbextension install /Users/patrickmulrooney/class/notebooks/pjmulroo/DSECapstone/autotime.py
'''
 
%load_ext autotime

In [3]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')


time: 29.5 ms


In [4]:
# Read config from local file
GMAP_API_KEY = None

sys.path.append('/Users/patrickmulrooney/Desktop/')
import capstone_config

_file, _ = capstone_config.getkeys()
print _file

/Users/patrickmulrooney/Downloads/endomondoHR/endoHRParsed_no_outliers.json
time: 4.89 ms


In [6]:
#time: 12min 44s
# Full dataset
#_file = "/Users/patrickmulrooney/Downloads/endomondoHR/endoHRParsed.json"
# Partial dataset
#_file = "/Users/patrickmulrooney/Downloads/endomondoHR/endoHRParsed_10k.json"

endoHR = []

with open(_file, 'r') as f:
    endoHR += ujson.load(f)

time: 14min 50s


In [46]:
dbname = 'endomondo2'
client = InfluxDBClient('127.0.0.1', 8086, 'root', 'root', dbname)

time: 115 ms


In [21]:
try:
    print("Drop database: " + dbname)
    client.drop_database(dbname)
except:
    pass

print("Create database: " + dbname)
client.create_database(dbname)

print("Create a retention policy")
client.create_retention_policy('awesome_policy', 'INF', 1, default=True)

Drop database: endomondo2
Create database: endomondo2
Create a retention policy
time: 34.1 ms


In [9]:
# # https://docs.influxdata.com/influxdb/v1.1/concepts/key_concepts/#database
# for _endoHR in log_progress(endoHR, every=1):
#     _points = []

#     _dict = {}
#     #_dict["measurement"] = "exercise"
#     _dict["measurement"] = _endoHR['sport']
#     _tags = {}
#     _tags["gender"] = _endoHR['gender']
#     _tags["workoutId"] = _endoHR['id']

#     #_tags["sport"] = _endoHR['sport']
#     #_dict["tags"]["url"] = _endoHR['url']
    
#     for _series in _endoHR['series']:
#         _dict["time"] = _series[0]
#         _dict["fields"] = {}
#         _dict["fields"]["userId"] = _endoHR['userId']
#         _dict["fields"]["ts"] = _series[0]

#         _dict["fields"]["heart_rate"] = _series[1]
#         _dict["fields"]["altitude"] = _series[2]
#         _dict["fields"]["latitude"] = _series[3]
#         _dict["fields"]["longitude"] = _series[4]
#         if _endoHR['speed_included']:
#             _dict["fields"]["speed"] = _series[5]

#         _points.append(copy.deepcopy(_dict))
    
#     client.write_points(_points, time_precision='s', tags=_tags)

time: 5.74 ms


In [ ]:
# https://docs.influxdata.com/influxdb/v1.1/concepts/key_concepts/#database
for _endoHR in log_progress(endoHR, every=1):
    _points = []

    _dict = {}
    #_dict["measurement"] = "exercise"
    _dict["measurement"] = _endoHR['sport']
    _tags = {}
    _tags["gender"] = _endoHR['gender']
    _tags["workoutId"] = _endoHR['id']
    _tags["userId"] = _endoHR['userId']


    #_tags["sport"] = _endoHR['sport']
    #_dict["tags"]["url"] = _endoHR['url']
    
    for _series in _endoHR['series']:
        _dict["time"] = _series[0]
        _dict["fields"] = {}
        #_dict["fields"]["ts"] = _series[0]

        _dict["fields"]["heart_rate"] = _series[1]
        _tags["altitude"] = _series[2]
        _tags["latitude"] = _series[3]
        _tags["longitude"] = _series[4]
        if _endoHR['speed_included']:
            _dict["fields"]["speed"] = _series[5]

        #_points.append(copy.deepcopy(_dict))
    
        client.write_points([_dict], time_precision='s', tags=_tags)

In [47]:
# https://docs.influxdata.com/influxdb/v1.1/concepts/key_concepts/#database
count = 1
_points = []
count2 = 18000
restart = 18000
for _endoHR in log_progress(endoHR[restart-150:], every=1):

    count2 += 1
    #if count2 < (restart-100):
    #    continue
    _dict = {}
    #_dict["measurement"] = "exercise"
    _dict["measurement"] = _endoHR['sport']
    _dict['tags'] = {}
    _dict['tags']["gender"] = _endoHR['gender']
    _dict['tags']["workoutId"] = _endoHR['id']
    _dict['tags']["userId"] = _endoHR['userId']


    #_tags["sport"] = _endoHR['sport']
    #_dict["tags"]["url"] = _endoHR['url']
    
    for _series in _endoHR['series']:
        _dict["time"] = _series[0]
        _dict["fields"] = {}
        #_dict["fields"]["ts"] = _series[0]

        _dict["fields"]["heart_rate"] = _series[1]
        _dict['tags']["altitude"] = _series[2]
        _dict['tags']["latitude"] = _series[3]
        _dict['tags']["longitude"] = _series[4]
        if _endoHR['speed_included']:
            _dict["fields"]["speed"] = _series[5]

        _points.append(copy.deepcopy(_dict))
    count += 1
    if count%6 == 0:
        try:
            client.write_points(_points, time_precision='s')
        except:
            client.write_points(_points, time_precision='s')

        count = 1
        _points = []
        restart = count2

ConnectionError: ('Connection aborted.', BadStatusLine("''",))

time: 1.22 s


In [ ]:
result = client.query('select count(*) from /.*/;')

In [63]:
result.raw['series'][0]['values'][0]

[u'1970-01-01T00:00:00Z',
 48607938,
 48607938,
 48607938,
 48607938,
 8740130,
 48607938,
 48607938]

time: 65.4 ms


In [55]:
sport_count = {}

for _endo in log_progress(endoHR, every=1):
    sport = _endo['sport']
    try:
        sport_count[sport] += len(_endo['series'])
    except:
        sport_count[sport] = len(_endo['series'])

time: 8min 5s


In [57]:
sport_count['run']

48659759

time: 2.93 ms
